<a href="https://colab.research.google.com/github/heyibad/genai-projects/blob/main/tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 830.2 kB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

GEMINI_API=userdata.get('GEMINI_API_KEY')

os.environ['GOOGLE_API_KEY']=GEMINI_API

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [4]:
from langchain_core.tools import tool
from typing import Union

@tool
def multiply(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Multiply two numbers. Use for questions like '5 times 3' or 'product of 12 and 4'."""
    return a * b

@tool
def divide(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Divide two numbers. Returns error for division by zero. Use for '10 divided by 2'."""
    if b == 0:
        return "Error: Division by zero"
    return a / b

@tool
def add(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Add two numbers. Use for 'sum of 5 and 3' or 'what is 12 + 4?'."""
    return a + b

@tool
def subtract(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    """Subtract two numbers. Use for '5 minus 3' or 'difference between 12 and 4'."""
    return a - b

tools = [multiply, divide, add, subtract]

In [5]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage



# Store conversation history in a list
conversation_history = [
    SystemMessage(
        content="You are a calculator. Only use your tools for math operations. "
                "For non-math questions, say: 'I can only perform calculations.' "
                "Always show your work using tools."
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
     verbose=True,  # Shows tool usage in console
    handle_parsing_errors=True,
    max_iterations=3
)

<ipython-input-5-ee65732b545f>:17: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [7]:
print("Calculator Bot: I can perform +, -, ×, ÷ operations.")
print("Type 'exit', 'end', or 'goodbye' to quit\n" + "="*40)

while True:
    user_input = input("\nYour input: ").strip()

    if user_input.lower() in ["exit", "end", "goodbye"]:
        print("\nCalculator Bot: Goodbye!")
        break

    # Add user message to history
    conversation_history.append(HumanMessage(content=user_input))

    try:
        response = agent.invoke({
            "input": user_input,
            "chat_history": conversation_history
        })

        # Add AI response to history
        ai_response = response["output"]
        conversation_history.append(AIMessage(content=ai_response))

        print("\n" + "="*40)
        print(f"User: {user_input}")
        print(f"Bot: {ai_response}")
        print("="*40)

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        conversation_history.append(AIMessage(content=error_msg))
        print(error_msg)

Calculator Bot: I can perform +, -, ×, ÷ operations.
Type 'exit', 'end', or 'goodbye' to quit

Your input: weather


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I am a calculator and cannot provide weather information."
}
```

> Finished chain.

User: weather
Bot: I am a calculator and cannot provide weather information.

Your input: 2 profuct 4


> Entering new AgentExecutor chain...
Thought: The user is asking for the product of 2 and 4. I should use the multiply tool.
Action:
```
{
  "action": "multiply",
  "action_input": {
    "a": 2,
    "b": 4
  }
}
```
Observation: 8
Thought:I have calculated the product.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The product of 2 and 4 is 8."
}
```

> Finished chain.

User: 2 profuct 4
Bot: The product of 2 and 4 is 8.

Your input: exit

Calculator Bot: Goodbye!


In [8]:
conversation_history

[SystemMessage(content="You are a calculator. Only use your tools for math operations. For non-math questions, say: 'I can only perform calculations.' Always show your work using tools.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='weather', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am a calculator and cannot provide weather information.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 profuct 4', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The product of 2 and 4 is 8.', additional_kwargs={}, response_metadata={})]